# LoL Draft Analyzer - ML Models

### Necessary imports

In [1]:
from transformers import *
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from tqdm import tqdm
import pickle
import warnings;
warnings.simplefilter('ignore')

### Importing Dataset

In [2]:
with open("x_train.pkl", "rb") as file:
    x_train = pickle.load(file)

with open("x_valid.pkl", "rb") as file:
    x_valid = pickle.load(file)

with open("x_test.pkl", "rb") as file:
    x_test = pickle.load(file)

with open("y_train.pkl", "rb") as file:
    y_train_dict = pickle.load(file)

with open("y_valid.pkl", "rb") as file:
    y_valid_dict = pickle.load(file)

with open("y_test.pkl", "rb") as file:
    y_test_dict = pickle.load(file)

## Logistic Regression

In [37]:
logistic_regression_pipe = Pipeline(steps=[
        ('RoleLineMerge', RoleLineMergeTransformer()),
        ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
        ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
        ('Model', LogisticRegression())
    ])

init called
init called
init called


## Sklearn Random Forest

In [38]:
random_forest_pipe = Pipeline(steps=[
    ('RoleLineMerge', RoleLineMergeTransformer()),
    ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
    ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
    ('Model', RandomForestClassifier(n_estimators=600, max_depth=8, n_jobs=-1))
])

init called
init called
init called


## XGBoost

In [39]:
xgboost_pipe = Pipeline(steps=[
        ('RoleLineMerge', RoleLineMergeTransformer()),
        ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
        ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
        ('lr', XGBClassifier(max_depth=3, n_jobs=-1))
    ])

init called
init called
init called


## Voting Classifier

In [40]:
from sklearn.ensemble import VotingClassifier

c1 = LogisticRegression()
c2 = RandomForestClassifier(n_estimators=600, max_depth=5, n_jobs=-1)
c3 = XGBClassifier(max_depth=3, n_jobs=-1)

voting_classifier_pipe = Pipeline(steps=[
        ('RoleLineMerge', RoleLineMergeTransformer()),
        ('DropUselessColumns', DropUselessColumnsTransformer(extra_input=[ExtraInputType.FIRST_BLOOD, ExtraInputType.FIRST_DRAGON, ExtraInputType.FIRST_HERALD, ExtraInputType.FIRST_TOWER])),
        ('MakeVector', MakeVectorTransformer(top_n=50, augmented=False)),
        ('vc', VotingClassifier(estimators=[('lr', c1), ('rf', c2), ('gnb', c3)], voting='soft'))
    ])


init called
init called
init called


In [41]:
a_y = y_train_dict['Win'].astype(float)

pipes = [logistic_regression_pipe.fit(x_train, a_y),
random_forest_pipe.fit(x_train, a_y),
xgboost_pipe.fit(x_train, a_y),
voting_classifier_pipe.fit(x_train, a_y)]

[21:40:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[21:41:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [42]:
for pipe in pipes:
    predictions = pipe.predict(x_test)
    print(accuracy_score(y_true=y_test_dict['Win'].astype(float), y_pred=predictions))

0.6782742681047765
0.6742681047765794
0.6755007704160246
0.6798151001540832
